In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
model = tf.keras.models.load_model('../model/twitter-spam')
model.summary()

2021-11-07 00:44:49.129692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-07 00:44:49.168520: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-07 00:44:49.168965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-07 00:44:49.169630: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 75, 12)            265884    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               275456    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 541,597
Trainable params: 541,597
Non-trainable params: 0
_________________________________________________________________


In [3]:
squid_tweets = pd.read_json('../dataset/squid-tweets/squid_tweets.json').rename(columns={0: 'tweets'})

In [4]:
squid_tweets['tweets']

0       #BTS, #TXT, And "#SquidGame" Nominated For 202...
1       tim pool watched squid game and somehow missed...
2              Squid Game is short for Squidward’s Gambit
3       Having tons of money doesn’t mean anything if ...
4       Squid game\n\nBAZINGA MV BUKAS NA\n@SB19Offici...
                              ...                        
1019                   Squid game https://t.co/8M8al6iWgb
1020       I gave squid game a shot and it’s pretty good.
1021    OMG this last ass scene from Squid Game is dra...
1022    @allkpopBuzz @allkpop At least Lebron watched ...
1023                      WHERE IS MY SQUID GAME COSTUME?
Name: tweets, Length: 1024, dtype: object

In [5]:
# Load tokenizer
import pickle
with open('../dataset/basic-tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

squid_seq = tokenizer.texts_to_sequences(squid_tweets['tweets'])
squid_pad = tf.keras.preprocessing.sequence.pad_sequences(squid_seq, padding='post', maxlen=71)

In [11]:
squid_pad[0]

array([ 1779, 20445,    12,     1,  3152,    10, 11303,     1,  1410,
        1318,     1,  5499,   789,  1760,     1,  5499,   789,  1760,
           1,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0],
      dtype=int32)

In [7]:
squid_tweets['is-spam'] = model.predict(squid_pad)

2021-11-07 00:45:08.873158: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-07 00:45:10.022151: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202


In [13]:
model.predict(squid_pad[0:2])

array([[1.658285e-04],
       [9.999201e-01]], dtype=float32)

In [8]:
squid_tweets['is-spam'] = [1 if i >= 0.5 else 0 for i in squid_tweets['is-spam']]
squid_tweets.tail()

,tweets,is-spam
1019,Squid game https://t.co/8M8al6iWgb,1
1020,I gave squid game a shot and it’s pretty good.,0
1021,OMG this last ass scene from Squid Game is dra...,0
1022,@allkpopBuzz @allkpop At least Lebron watched ...,0
1023,WHERE IS MY SQUID GAME COSTUME?,0


In [34]:
def pad_seq(text):
    global tokenizer
    seq = tokenizer.texts_to_sequences([text])
    pad = tf.keras.preprocessing.sequence.pad_sequences(seq, padding='post', maxlen=71)
    return pad

In [31]:
text = 'Hey, do do you know that we are selling new stocks? Go check our website now!'
seq = tokenizer.texts_to_sequences([text])[0]
pad = 
seq

[588, 56, 56, 16, 82, 24, 33, 27, 1192, 41, 4134, 96, 275, 69, 1314, 66]

In [35]:
# Cases
model.predict(pad_seq('Hey, do you know that we are selling new stocks? Go check our website now!'))


array([[1.1930588e-13]], dtype=float32)

In [37]:
model.predict(pad_seq('Good Morning'))

array([[6.09603e-08]], dtype=float32)

In [46]:
squid_tweets.to_excel('../dataset/squid-tweets/squid-dataset.xlsx')